<a href="https://colab.research.google.com/github/simonopravil/nn_to_al-/blob/main/01_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
!pip install eemont
!pip install geemap
!pip install eerepr --ignore-requires-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import libraries
import ee, eemont, geemap, eerepr
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import geopandas as gpd

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xx-C0OAO65_KJtPcMOlp_PGYaTKpRg5ToVoGyQH0l2g&tc=GAweWUzCQs5XUrUln6DSkEA3MBRCmYjrbhzNQL7YjcI&cc=2jSbaq26Bawp7zcSkb7P6Is11dwclBJIgKQzRszM23M

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvvY6YOwpUWCAB7oSHkUD39OGOE1119JgCzho9KvyffqiP1YFI5XTCE

Successfully saved authorization token.


In [ ]:
samples_shp = '/content/drive/MyDrive/PhD/Projects/CroplandGrasslands/SHP/AOI.shp'
tat = gpd.read_file(samples_shp)
samples = geemap.shp_to_ee(samples_shp)

In [ ]:
# Here we define some usefull function that we can map over ImageColelction
def clp (image, geometry):
  """
  This function return cliped image for 
  every image in ImageCollection
  """
  return image.clip(geometry)

In [ ]:
tat

,id,geometry
0,1,"POLYGON ((240616.544 5365117.825, 285247.258 5..."
1,2,"POLYGON ((375378.117 5445829.864, 421168.072 5..."
2,3,"POLYGON ((559518.660 5394995.142, 582768.654 5..."
3,4,"POLYGON ((577448.026 5469304.461, 578670.024 5..."


In [ ]:
for i in tat.id:
  select = tat[tat['id'] == i].to_crs('EPSG:4326')
  convert = geemap.geopandas_to_ee(select)

  S2_items = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(convert) #Filter ImageCollection by Spatial Extent
      .filterDate('2018-04-01','2018-10-30') #Filter ImageCollection by time
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25))
      .maskClouds()
      .select(['B2', 'B3', 'B4', 'B8']))
  
  S2_median = S2_items.median().clip(convert.geometry())
  S2_first = S2_items.first().clip(convert.geometry())

  task = ee.batch.Export.image.toDrive(**{
    'image': S2_median,
    'description': 's2cnn{zone}'.format(zone = i),
    'folder':'geeExport',
    'scale': 10
    })
  task.start()